In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import seaborn as sns
from pyDOE import lhs
import random
import os
import time

plt.rcParams.update({'font.size':18})

In [2]:
def seed_torch(seed=1024):
    #     random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 为了禁止hash随机化，使得实验可复现
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [3]:
domain = (0, 10, 0, 1,0, 1)
tmin, tmax, xmin, xmax, ymin, ymax = domain
mlp_layers = [3] + [20]*4 + [1]
# adam_iters = 40000
adam_iters = 20000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = r'./model'
train_info_path = r'./'
if not os.path.exists(model_path):
    os.mkdir(model_path)

In [4]:
class Dataset:
    def __init__(self, domain):
        self.domain = domain
        self.lb = np.array([tmin, xmin, ymin])
        self.ub = np.array([tmax, xmax, ymax])
        self.N_max = 20000
        self.x_range = (xmin, xmax)
        self.y_range = (ymin, ymax)
        self.t_range = (tmin, tmax)

    def train_data(self, verbose=None):
        tmin, tmax, xmin, xmax, ymin, ymax = self.domain
#         # 内部点采样
#         t_res = np.linspace(tmin, tmax, 50)
#         x_res = np.linspace(xmin, xmax, 40)
#         y_res = np.linspace(ymin, ymax, 40)
#         X_res = self.sample_xy(x_res, y_res)
        
#         N = X_res.shape[0]  # N=1600
#         T = t_res.shape[0]  # T=50
        
#         XX = np.tile(X_res[:,0:1], (1,T))  # N,T
#         YY = np.tile(X_res[:,1:2], (1,T))  # N,T
#         TT = np.tile(t_res.T, (N,1))  # N,T
        
#         x = XX.flatten()[:, None]  # NT x 1
#         y = YY.flatten()[:, None]  # NT x 1
#         t = TT.flatten()[:, None]  # NT x 1
        
#         X_res = np.concatenate([t.reshape((-1, 1)), x.reshape((-1, 1)), y.reshape((-1, 1))], axis=1)
# #         print(X_res.shape)
        
        # 新的采样方法
        X = torch.Tensor(self.N_max, 1).uniform_(*self.x_range)
        Y = torch.Tensor(self.N_max, 1).uniform_(*self.y_range)
        T = torch.Tensor(self.N_max, 1).uniform_(*self.t_range)
        
        idx = torch.randperm(self.N_max)
        idx = idx[:10000]
        X_res = torch.cat([T[idx], X[idx], Y[idx]], dim=1)

        # 初始点采样
#         t_ics = np.ones((10,1))*tmin
#         x_ics = np.linspace(xmin, xmax, 20)
#         y_ics = np.linspace(ymin, ymax, 20)
#         X_ics = self.sample_xy(x_ics, y_ics)
        
#         N = X_ics.shape[0]  # N=1600
#         T = t_ics.shape[0]  # T=50
        
#         XX = np.tile(X_ics[:,0:1], (1,T))  # N,T
#         YY = np.tile(X_ics[:,1:2], (1,T))  # N,T
#         TT = np.tile(t_ics.T, (N,1))  # N,T
        
#         x = XX.flatten()[:, None]  # NT x 1
#         y = YY.flatten()[:, None]  # NT x 1
#         t = TT.flatten()[:, None]  # NT x 1
        
#         X_ics = np.concatenate([t.reshape((-1, 1)), x.reshape((-1, 1)), y.reshape((-1, 1))], axis=1)

        # 新的采样方法
        X = torch.Tensor(self.N_max, 1).uniform_(*self.x_range)
        Y = torch.Tensor(self.N_max, 1).uniform_(*self.y_range)
        T = torch.Tensor(self.N_max, 1).fill_(0)
        
        idx = torch.randperm(self.N_max)
        idx = idx[:10000]
        X_ics = torch.cat([T[idx], X[idx], Y[idx]], dim=1)
        
        
        # X_ics = np.concatenate([t_ics.reshape((-1,1)), x_ics.reshape((-1, 1)), y_ics.reshape((-1, 1))], axis=1)
        u_ics = self.u_ics_sol(X_ics)
        
#         # 降低一维
#         X_res = np.squeeze(X_res)
#         X_ics = np.squeeze(X_ics)
#         u_ics = np.squeeze(u_ics)

        return X_res, X_ics, u_ics


    def sample_xy(self, x, y):
        xx, yy = np.meshgrid(x, y)
        X = np.concatenate([xx.reshape((-1, 1)), yy.reshape((-1, 1))], axis=1)
        return X
    
#     def sample_xyz(self, t,x,y):
#         tt, xx, yy = np.meshgrid(t,x,y)
#         X = np.concatenate([tt[:,:,:,None],xx[:,:,:,None],yy[:,:,:,None]], axis=1)
#         return X
    
    def u_ics_sol(self, X):
        return np.tanh((0.35-np.sqrt((X[:,1]-0.5)**2 + (X[:,2]-0.5)**2))/(2*0.025))
    
    def lhs_sample_xy(self, n=200):
        X = (self.ub - self.lb) * lhs(3, n) + self.lb
        print(X.shape)
        return X
    
dataset = Dataset(domain)
# 内部点与边界点
X_res, X_ics, u_ics = dataset.train_data()
print(X_res.shape, X_ics.shape,u_ics.shape)
# dataset.lhs_sample_xy()

torch.Size([10000, 3]) torch.Size([10000, 3]) torch.Size([10000])


## DNN

In [5]:
class MLP(nn.Module):
    def __init__(self, mlp_layers):
        super(MLP, self).__init__()
        
        self.model = nn.Sequential()
        for i in range(len(mlp_layers)-2):
            layer = nn.Sequential()
            layer.add_module(f'fc{i}', nn.Linear(mlp_layers[i], mlp_layers[i+1], bias=True))
            layer.add_module(f'act{i}', nn.Tanh())
            self.model.add_module(f'layer{i}', layer)

        last_layer = nn.Sequential()
        last_layer.add_module(f'fc{len(mlp_layers)-2}', nn.Linear(mlp_layers[-2], mlp_layers[-1], bias=False))
        self.model.add_module(f'layer{len(mlp_layers)-2}', last_layer)
        
        for param in self.parameters():
            if len(param.shape) > 1:
                nn.init.kaiming_normal_(param)
    
    def forward(self, X):
        return self.model(X)

    
    
mlp = MLP(mlp_layers)
mlp

MLP(
  (model): Sequential(
    (layer0): Sequential(
      (fc0): Linear(in_features=3, out_features=20, bias=True)
      (act0): Tanh()
    )
    (layer1): Sequential(
      (fc1): Linear(in_features=20, out_features=20, bias=True)
      (act1): Tanh()
    )
    (layer2): Sequential(
      (fc2): Linear(in_features=20, out_features=20, bias=True)
      (act2): Tanh()
    )
    (layer3): Sequential(
      (fc3): Linear(in_features=20, out_features=20, bias=True)
      (act3): Tanh()
    )
    (layer4): Sequential(
      (fc4): Linear(in_features=20, out_features=1, bias=False)
    )
  )
)

## 主干网络

In [6]:
def grad(outputs, inputs):
    return torch.autograd.grad(outputs, inputs,
                               grad_outputs=torch.ones_like(outputs),
                               create_graph=True, 
                               retain_graph=True)

In [7]:
class PINN(nn.Module):
    def __init__(self, backbone, mu=None, sigma=None):
        super(PINN, self).__init__()
        self.backbone = backbone

    def forward(self, X_res, X_ics, u_ics):
        
        loss_res = torch.mean(self.net_f(X_res) ** 2)
        loss_ics = torch.mean((self.net_u(X_ics)-u_ics) ** 2)
        return loss_res, loss_ics
    
    def net_u(self, X):
        return self.backbone(X)
    
    def net_u_x(self,X):
        X.requires_grad_(True)
        u = self.net_u(X)
        
        # 求梯度
        grad_u = grad(u, X)[0]
        u_x = grad_u[:, [1]]
        return u_x

    def net_f(self, X):
        X.requires_grad_(True)

        u = self.net_u(X)
#         print(u.shape)
        # 求梯度
        grad_u = grad(u, X)[0]
        u_t = grad_u[:, [0]]
        u_x = grad_u[:, [1]]
        u_tt = grad(u_t, X)[0][:, [0]]
        u_xx = grad(u_x, X)[0][:, [1]]
        
        eps = 0.025
        lam = 10
        
        f = u_t - (eps**2*u_xx - u**3 + u)*lam
        return f  

pinn = PINN(mlp)

## Resample策略

In [8]:
def easy_resample(dataset, net_f, device=torch.device('cuda'), n_total_points=4000, n_added_points=500):
    # 生成待采样的点集
    X_resam = dataset.lhs_sample_xy(n=n_total_points)
    X_resam = torch.from_numpy(X_resam).float().to(device)
    # 获得residule
    f = net_f(X_resam)
    f = f.detach().cpu().numpy()
    # 依loss_res的值排序得到索引
    idx = np.argsort(abs(f).flatten())[-n_added_points:]
    return X_resam[idx].detach()

## Adam

In [9]:
dataset = Dataset(domain)
X_res, X_ics, u_ics = dataset.train_data()

# X_res = torch.from_numpy(X_res).float().to(device)
X_res = X_res.float().to(device)
# X_bcs_l = torch.from_numpy(X_bcs_l).float().to(device)
# X_bcs_u = torch.from_numpy(X_bcs_u).float().to(device)
X_ics = X_ics.float().to(device)
u_ics = u_ics.float().to(device)
# X_ics = torch.from_numpy(X_ics).float().to(device)
# u_ics = torch.from_numpy(u_ics).float().to(device)

mu = X_res.mean(dim=0)
sigma = X_res.std(dim=0)  # 求样本标准差

backbone = MLP(mlp_layers)  # 主干网络
pinn = PINN(backbone, mu, sigma).to(device)

optimizer_adam = optim.Adam(pinn.backbone.parameters(), lr=1e-3)

lr_sche = optim.lr_scheduler.ExponentialLR(optimizer_adam, gamma=0.8)  # 指数衰减学习率

logger = {
    "loss": [], 
    "loss_res": [],
    "loss_ics": [],
    "loss_bcs": [],
    "loss_bcs_t": [],
    "iter": [],
    "mu": mu,
    "sigma": sigma
}
best_loss = 1e9

# 训练
start_time = time.time()
for it in range(adam_iters):
    pinn.train()
    pinn.zero_grad()
    
    loss_res, loss_ics = pinn(X_res, X_ics, u_ics)
    loss = loss_res + loss_ics*100
    
    loss.backward()
    optimizer_adam.step()
    
    if (it + 1) % 100 == 0:
        # 保存loss信息
        pinn.train(False)
        loss_res_valid, loss_ics_valid = pinn(X_res, X_ics, u_ics)
        loss_valid = loss_res_valid + loss_ics_valid 
        
        logger["loss"].append(loss_valid.item())
        logger["loss_res"].append(loss_res_valid.item())
        logger["loss_ics"].append(loss_ics_valid.item())
        logger["iter"].append(it+1)
        
        
        # 保存训练loss最低的模型
        if loss_valid.item() < best_loss:
            model_state = {'iter': it+1, 'backbone_state': pinn.backbone.state_dict()}
            torch.save(model_state, os.path.join(model_path, 'pinn_adam.pth'))
            best_loss = loss_valid.item()
        
        if (it + 1) % 500 == 0:
            # 保存并打印训练日志
            info = f'Iter # {it+1:6d}/{adam_iters}\t' + \
                f'loss:{loss.item():.2e}, loss_r:{loss_res.item():.2e}, loss_i:{loss_ics.item():.2e}  ' + \
                f'Valid # loss:{loss_valid.item():.2e}, loss_r:{loss_res_valid.item():.2e}, loss_i:{loss_ics_valid.item():.2e}'
            with open(train_info_path + 'train_info.txt', 'a') as f:
                f.write(info + '\n')
            print(info)
            
        # 衰减学习率
        if (it + 1) % 4000 == 0:
            lr_sche.step()
            
    if (it + 1) % 1000 == 0:
        pinn.zero_grad()
        pinn.eval()
        # 进行重采样
#         print(X_res.shape)
        X_resam1 = easy_resample(dataset, pinn.net_f, device = device).float()
        # 拼接数据
        X_res = torch.cat([X_res, X_resam1], dim=0)
#         print(X_res.shape)
        

Iter #    500/20000	loss:3.05e+01, loss_r:1.10e+00, loss_i:2.94e-01  Valid # loss:1.39e+00, loss_r:1.10e+00, loss_i:2.94e-01
Iter #   1000/20000	loss:3.04e+01, loss_r:9.70e-01, loss_i:2.94e-01  Valid # loss:1.26e+00, loss_r:9.70e-01, loss_i:2.94e-01
(4000, 3)
Iter #   1500/20000	loss:3.06e+01, loss_r:1.23e+00, loss_i:2.94e-01  Valid # loss:1.53e+00, loss_r:1.23e+00, loss_i:2.94e-01
Iter #   2000/20000	loss:3.06e+01, loss_r:1.20e+00, loss_i:2.94e-01  Valid # loss:1.50e+00, loss_r:1.20e+00, loss_i:2.94e-01
(4000, 3)
Iter #   2500/20000	loss:3.08e+01, loss_r:1.45e+00, loss_i:2.94e-01  Valid # loss:1.74e+00, loss_r:1.45e+00, loss_i:2.94e-01
Iter #   3000/20000	loss:3.08e+01, loss_r:1.42e+00, loss_i:2.94e-01  Valid # loss:1.72e+00, loss_r:1.42e+00, loss_i:2.94e-01
(4000, 3)
Iter #   3500/20000	loss:3.10e+01, loss_r:1.64e+00, loss_i:2.94e-01  Valid # loss:1.93e+00, loss_r:1.64e+00, loss_i:2.94e-01
Iter #   4000/20000	loss:3.10e+01, loss_r:1.60e+00, loss_i:2.94e-01  Valid # loss:1.90e+00, los

KeyboardInterrupt: 

In [ ]:
np.save("./logger.npy", logger)

In [ ]:
logger = np.load("./logger.npy", allow_pickle=True).item()
k = 2
with sns.axes_style("darkgrid"):
    plt.figure(figsize=(9, 7))
    plt.subplot(111)
    # plt.plot(logger["iter"][::k], logger["loss"][::k], label=r"$L$")
    plt.plot(logger["iter"][::k], logger["loss_res"][::k], label=r"$\mathcal{L}_{r}$", linewidth=3)
    plt.plot(logger["iter"][::k], logger["loss_ics"][::k], label=r"$\mathcal{L}_{ics}$", linewidth=3)
    plt.legend()
    plt.xticks([0, 5000, 10000, 15000, 20000])
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.yscale('log')
    plt.savefig('loss.png', dpi=100)
    plt.show()

In [ ]:
# t_res = np.linspace(tmin, tmax, 100)
t_ics = np.ones((40,1))*tmin
x_res = np.linspace(xmin, xmax, 100)
y_res = np.linspace(ymin, ymax, 100)
xx, yy = np.meshgrid(x_res,y_res)
X = np.concatenate([x_res.reshape(-1, 1), y_res.reshape(-1, 1)], axis=1)



N = X_res.shape[0]  # N=1600
T = t_res.shape[0]  # T=50

XX = np.tile(X_res[:,0:1], (1,T))  # N,T
YY = np.tile(X_res[:,1:2], (1,T))  # N,T
TT = np.tile(t_res.T, (N,1))  # N,T

snap = np.array([20])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]

x_star = torch.tensor(x_star)
y_star = torch.tensor(y_star)
t_star = torch.tensor(t_star)

X = torch.cat([x_star,y_star,t_star], dim=1)
X = torch.tensor(X)


u_pred = pinn.net_u(X).detach().cpu().numpy()
X_res = torch.from_numpy(X_res).cpu().numpy()
print(xx.shape)
u_pred.shape


In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = plt.subplot(1, 1, 1)
plt.pcolor(xx, yy, u_pred[0,:,].reshape(xx.shape), cmap='jet', vmin = -1, vmax =1)
plt.colorbar()
# plt.clim([-1., 1.])
plt.xlabel('$t$')
plt.ylabel('$x$')
ax.set_xlim([0, 1])
ax.set_ylim([-1, 1])
ax.set_xticks(np.linspace(0, 1, 5))
ax.set_yticks(np.linspace(-1, 1, 5))
plt.title(r'Predicted $u(t,x)$')
ax.set_aspect(1./ax.get_data_ratio())
plt.tight_layout()
plt.savefig('AC_pred_viridis.png', dpi=100)